In [19]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import glob
import os

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, Softmax

# Build the LSTM model without GlobalAveragePooling1D
def build_lstm_model(sequence_length=80, feature_size=2048):
    model = Sequential()
    model.add(LSTM(2048, input_shape=(sequence_length, feature_size), return_sequences=False))
    model.add(Dropout(0.4))
    model.add(Dense(2048, activation='relu'))
    model.add(Dense(2048, activation='relu'))
    model.add(Dense(2, activation='softmax'))  # Assuming binary classification (real/fake)
    return model


In [3]:
# Sequence length is 80 and each frame has 2048 features
input_shape = (80, 2048)
lstm_model = build_lstm_model(sequence_length=80, feature_size=2048)
lstm_model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 2048)              33562624  
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 2048)              4196352   
                                                                 
 dense_1 (Dense)             (None, 2048)              4196352   
                                                                 
 dense_2 (Dense)             (None, 2)                 4098      
                                                                 
Total params: 41,959,426
Trainable params: 41,959,426
Non-trainable params: 0
_________________________________________________________________


In [4]:
# Sequence length is 80 and each frame has 2048 features
input_shape = (80, 2048)
lstm_model = build_lstm_model(sequence_length=80, feature_size=2048)
lstm_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 2048)              33562624  
                                                                 
 dropout_1 (Dropout)         (None, 2048)              0         
                                                                 
 dense_3 (Dense)             (None, 2048)              4196352   
                                                                 
 dense_4 (Dense)             (None, 2048)              4196352   
                                                                 
 dense_5 (Dense)             (None, 2)                 4098      
                                                                 
Total params: 41,959,426
Trainable params: 41,959,426
Non-trainable params: 0
_________________________________________________________________


In [12]:
# Mock function to load features from .npy files
def load_features(directory):
    feature_files = glob.glob(os.path.join(directory, '*.npy'))
    features = []
    
    for file in feature_files:
        feature_array = np.load(file)
        features.append(feature_array)
    
    # Stack the features along the first axis (batch size)
    return np.vstack(features)

In [13]:
# Load training and validation features
train_features_real = load_features(r'C:\Users\install\Documents\Project\features\train\real')
train_features_fake = load_features(r'C:\Users\install\Documents\Project\features\train\fake')

In [14]:
# Stack real and fake features and create labels
train_features = np.vstack((train_features_real, train_features_fake))
train_labels = np.array([1] * train_features_real.shape[0] + [0] * train_features_fake.shape[0])  # 1 for real, 0 for fake

In [15]:
valid_features_real = load_features(r'C:\Users\install\Documents\Project\features\valid\real')
valid_features_fake = load_features(r'C:\Users\install\Documents\Project\features\valid\fake')

In [16]:
# Stack validation features and create labels
valid_features = np.vstack((valid_features_real, valid_features_fake))
valid_labels = np.array([1] * valid_features_real.shape[0] + [0] * valid_features_fake.shape[0])  # 1 for real, 0 for fake

In [27]:
# Compile the model
lstm_model.compile(
    optimizer=Adam(learning_rate=0.0001, decay=0.001),
    loss=SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [20]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

In [23]:
# Assume train_features is a 2D array of shape (num_samples, 2048)
# Reshape it into (num_samples, 80, 2048)
# This assumes that each sample has enough frames (80) for the LSTM.

num_samples = train_features.shape[0] // 80  # Number of complete sequences you can create
train_features_reshaped = train_features[:num_samples * 80].reshape(num_samples, 80, 2048)

# Repeat for validation features
num_valid_samples = valid_features.shape[0] // 80
valid_features_reshaped = valid_features[:num_valid_samples * 80].reshape(num_valid_samples, 80, 2048)

# Ensure that the labels correspond to the reshaped features
train_labels_reshaped = np.repeat(train_labels[:num_samples * 80], 80)[:train_features_reshaped.shape[0]]
valid_labels_reshaped = np.repeat(valid_labels[:num_valid_samples * 80], 80)[:valid_features_reshaped.shape[0]]


In [25]:
print("Train features shape:", train_features_reshaped.shape)
print("Train labels shape:", train_labels_reshaped.shape)
print("Validation features shape:", valid_features_reshaped.shape)
print("Validation labels shape:", valid_labels_reshaped.shape)

Train features shape: (39, 80, 2048)
Train labels shape: (39,)
Validation features shape: (16, 80, 2048)
Validation labels shape: (16,)


In [26]:
print("Train Features NaNs:", np.isnan(train_features_reshaped).any())
print("Train Features Infs:", np.isinf(train_features_reshaped).any())
print("Validation Features NaNs:", np.isnan(valid_features_reshaped).any())
print("Validation Features Infs:", np.isinf(valid_features_reshaped).any())

Train Features NaNs: False
Train Features Infs: False
Validation Features NaNs: False
Validation Features Infs: False


In [28]:
# Train the model using reshaped features
lstm_model.fit(
    train_features_reshaped, train_labels_reshaped,
    validation_data=(valid_features_reshaped, valid_labels_reshaped),
    epochs=15,
    callbacks=[early_stopping]
)


Epoch 1/15
2/2 [==============================] - 2s 797ms/step - loss: 0.0131 - accuracy: 1.0000 - val_loss: 5.9232e-06 - val_accuracy: 1.0000
Epoch 2/15
2/2 [==============================] - 0s 116ms/step - loss: 2.7785e-06 - accuracy: 1.0000 - val_loss: 1.4901e-08 - val_accuracy: 1.0000
Epoch 3/15
2/2 [==============================] - 0s 239ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/15
2/2 [==============================] - 0s 52ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/15
2/2 [==============================] - 0s 66ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/15
2/2 [==============================] - 0s 68ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/15
2/2 [==============================] - 0s 58ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000